# OpenBnB MCP சர்வர் ஒருங்கிணைப்புடன் செமாண்டிக் கர்னல்

இந்த நோட்புக் மெய்நிகர் Airbnb தங்குமிடங்களை தேட OpenBnB MCP சர்வருடன் செமாண்டிக் கர்னலை MCPStdioPlugin ஐ பயன்படுத்தி எப்படி பயன்படுவதை காட்சிப்படுத்துகிறது. LLM அணுகலுக்கு, இது Microsoft Foundry ஐ பயன்படுத்துகிறது. உங்கள் சூழல் மாறிலிகளை அமைக்க, நீங்கள் [அமைக்க கற்கை](/00-course-setup/README.md) ஐ பின்பற்றலாம்.


## தேவையான தொகுப்புகளை இறக்குமதி செய்யுங்கள்


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP பதிவி இணைப்பை உருவாக்குதல்

நாம் MCPStdioPlugin ஐ பயன்படுத்தி [OpenBnB MCP சர்வர்](https://github.com/openbnb-org/mcp-server-airbnb) உடன் இணைக்க போகிறோம். இந்த சர்வர் @openbnb/mcp-server-airbnb தொகுப்பின் மூலம் Airbnb தேடல் செயல்பாட்டை வழங்குகிறது.


## கிளையண்டை உருவாக்குவது

இந்த மாதிரியில், நாங்கள் எங்கள் LLM அணுகலுக்கு Microsoft Foundry-ஐப் பயன்படுத்தவுள்ளோம். உங்கள் சூழல் மாறிலிகள் சரியாக அமைக்கப்பட்டுள்ளதா என்பதை உறுதியளிக்கவும்.


## சூழல் கட்டமைப்பு

Azure OpenAI அமைப்புகளை கட்டமைக்கவும். கீழ்காணும் சூழல் மாறிலிகள் அமைக்கப்பட்டுள்ளதே என உறுதிசெய்யவும்:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ஒருங்கிணைப்பை புரிந்துகொள்வது

இந்த நோட்புக் உண்மையான Airbnb தேடல் செயல்பாட்டை வழங்கும் **உண்மையான OpenBnB MCP சர்வருடன்** இணைக்கிறது.

### அது எப்படி பணியாற்றுகிறது:

1. **MCPStdioPlugin**: MCP சர்வருடன் இயல்புநிலை உள்ளீடு/வெளியீடு தொடர்பை பயன்படுத்துகிறது
2. **உண்மையான NPM தொகுப்பு**: npx மூலம் `@openbnb/mcp-server-airbnb` ஐ பதிவிறக்கம் செய்து இயக்குகிறது
3. **நேரடி தரவு**: அவர்களின் API களிலிருந்து உண்மையான Airbnb சொத்துகளின் தரவை திரும்ப வழங்குகிறது
4. **செயல்பாடு கண்டறிதல்**: முகவர் தானாக MCP சர்வரிலிருந்து கிடைக்கும் செயல்பாடுகளை கண்டறிகிறது

### கிடைக்கும் செயல்பாடுகள்:

OpenBnB MCP சர்வர் பொதுவாக வழங்குகிறது:
- **search_listings** - இடம் மற்றும் அடையாளக்குறிகளின் மூலம் Airbnb சொத்துகளை தேடுகிறது
- **get_listing_details** - குறிப்பிட்ட சொத்துகளுக்கான விரிவான தகவல்களை பெறுகிறது
- **check_availability** - குறிப்பிட்ட தேதிகளுக்கான கிடைக்கும் நிலையை சரிபார்க்கிறது
- **get_reviews** - சொத்துகளுக்கான மதிப்பாய்வுகளை பெறுகிறது
- **get_host_info** - சொத்து அனதிக்காரர்களின் தகவல்களை பெறுகிறது

### முன் தேவைகள்:

- உங்கள் கணினியில் **Node.js** நிறுவப்பட்டிருக்க வேண்டும்
- MCP சர்வர் தொகுப்பை பதிவிறக்கம் செய்ய **இணைய இணைப்பு** வேண்டும்
- **NPX** கிடைய வேண்டும் (Node.js உடன் வருகிறது)

### இணைப்பை சோதனை செய்தல்:

MCP சர்வரை கையடக்கமாக இயக்கி சோதிக்க முடியும்:
```bash
npx -y @openbnb/mcp-server-airbnb
```

இது OpenBnB MCP சர்வரை பதிவிறக்கம் செய்து துவங்கும், அதன் பிறகு Semantic Kernel உண்மையான Airbnb தரவிற்காக அதனுடன் இணைகிறது.


## OpenBnB MCP சர்வரை கொண்டு முகவரியை இயக்குதல்

இப்போது, ஸ்டாக்ஹோம் நகரில் 2 பெரியவர்கள் மற்றும் 1 குழந்தைக்கு ஏற்ப உட்பட்ட உண்மையான Airbnb தங்குமிடங்களை தேட OpenBnB MCP சர்வருடன் இணைக்கப்படும் AI முகவரியை இயக்குகிறோம். தேடல் அளவுருக்களை மாற்ற `user_inputs` பட்டியலை மாற்றலாம்.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# சுருக்கம்
வாழ்த்துக்கள்! நீங்கள் Model Context Protocol (MCP) மூலம் உண்மையான உலகிலுள்ள தங்க இடம் தேடலை ஒருங்கிணைக்கும் ஒரு AI முகவரியை வெற்றிகரமாக உருவாக்கியுள்ளீர்கள்:

## பயன்படுத்தப்பட்ட தொழில்நுட்பங்கள்:
- Semantic Kernel - Azure OpenAI உடன் நுண்ணறிவு முகவரிகளை உருவாக்க
- Microsoft Foundry - LLM திறன்கள் மற்றும் உரையாடல் முடிவுகளுக்கான
- MCP (Model Context Protocol) - நிலையான கருவி ஒருங்கிணைக்க
- OpenBnB MCP Server - உண்மையான Airbnb தேடல் செயல்பாட்டிற்கு
- Node.js/NPX - வெளிப்புற MCP சேவையகத்தை இயக்க

## நீங்கள் கற்றுக்கொண்டது:
- MCP ஒருங்கிணைப்பு: Semantic Kernel முகவர்கள் வெளியுற MCP சேவையகங்களுடன் இணைப்பு
- நேரடி தரவு அணுகல்: நேரடி API-களின் மூலம் உண்மையான Airbnb சொத்துக்களை தேடுதல்
- நெறிமுறை தொடர்பு: முகவர் மற்றும் MCP சேவையகத்துக்கிடையில் stdio தொடர்பு பயன்படுத்தல்
- செயல்பாட்டு கண்டுபிடிப்பு: MCP சேவையகங்களிலிருந்து கிடைக்கும் செயல்பாடுகளை தானாக கண்டுபிடித்தல்
- ஓடும் பதில்கள்: செயல்பாட்டு அழைப்புகளை நேரடி பதிவு செய்தல் மற்றும் பதிவு செய்தல்
- HTML காட்சி: முகவர் பதில்களை அலங்கரிக்கப்பட்ட அட்டவணைகள் மற்றும் இதயமுள்ள காட்சிகளுடன் வடிவமைத்தல்

## அடுத்த படிகள்:
- கூடுதல் MCP சேவையகங்களை ஒருங்கிணைக்க (வானிலை, விமானங்கள், உணவகங்கள்)
- MCP மற்றும் A2A நெறிமுறைகளை இணைக்கும் பன்முகவர் அமைப்பை உருவாக்க
- உங்கள் சொந்த தரவுக்காட்டிகளில் தனியான MCP சேவையகங்களை உருவாக்க
- உரையாடல் நினைவாற்றலை அமுல்படுத்த
- MCP சேவையக ஒருங்கிணைப்புடன் Azure Functions-க்கு முகவரியை நிறுவ
- பயனர் அங்கீகாரம் மற்றும் முன்பதிவு திறன்களை சேர்க்க


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**தயாரிப்பு அறிவிப்பு**:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நம்பகத்தன்மைக்கு நாங்கள் காத்திருக்கிறோம் என்பதுடன், தானாக மொழிபெயர்ப்புகள் பிழைகள் அல்லது தவறுகள் உள்ளடக்கப்படலாம் என்பதை நினைவில் கொள்ளவும். சொந்த நாட்டின் மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களின் படியில், தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் ஏற்படும் எந்தவொன்று தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பாக இருக்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
